# Creative Nation GTR Analysis

This notebook analyses Gateway to Reseaerch data for the Creative Nation report.

We start with a list of projects classified by Discipline (based on the Arloesiadur analysis) and we classify those projects into different categories based on the nature of the organisations involved:

`````
                Creative 
                organisation
                Y     N
=============================
Creative    Y  CI    CE 
discipline  N  EK    OTHER
=============================

```

* ```CI```: A creative organisation engaging with a creative discipline. 
* ```CE```: A non-creative organisation engaging with a creative discipline. 
* ```EK```: A creative organisation engaging with a non-creative discipline. 
* ```OTHER```: A non-creative organisation engaging with a non-creative project. 

**Definitions**

We define **creative organisations** based on DCMS SIC codes

How do we deal with creative non-commercial organisations and creative stakeholders e.g. BBC, Arts Council, Design Council etc.? Use members of the Creative Industries Council or Federation

We define **creative disciplines** based on their creative intensity (the relative importance of creative businesses in the discipline). 

**Activities**

1. Load GtR data 
   * Are we doing anything with Innovate UK and other data sources?
2. Query org names vs Companies House
3. Identify creative non-commercial orgs
4. Define creative disciplines
5. Classify projects
6. Consider options for SNA. 




## Preamble

In [1]:
#Imports

% matplotlib inline

#Use this script to format some of the code as markdown
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#Utilities
import os
import urllib3
import requests
import zipfile
import json
import pickle
import datetime
import sys
import http.client, urllib.request, urllib.parse, urllib.error, base64
import re


#Data 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations

import http.client, urllib.request, urllib.parse, urllib.error, base64

#Analytical
import networkx as nx
import community

#Otheer
import ratelim
from bs4 import BeautifulSoup
import chwrapper


#Paths
top = os.path.dirname(os.getcwd())

#External data (to download the GRID database)
ext_data = os.path.join(top,'data/external')

#Interim data (to place seed etc)
int_data = os.path.join(top,'data/interim')

#Figures
fig_path = os.path.join(top,'reports/figures')


#Get date for saving files
today = datetime.datetime.today()

today_str = "_".join([str(x) for x in [today.day,today.month,today.year]])

#Increase number of recursions to save a pickle file later
sys.setrecursionlimit(10000)

#Token
#Load token
with open(top+'/keys.json','r') as infile:
    tokens = json.load(infile)
ch_token = tokens['ch']



In [4]:
#Write the class to do this


class GTR_org():
    '''
    We use this class to process a set of companies from Gateway to Research.
    This involves:
        Querying CH with the company name
        Disambiguate results with the GTR company postcode
        Matching the results with the CH datadump using CH number and obtaining the SIC code
        Classifying companies into CE sectors based on their SIC code
    
    '''
    
    def __init__(self,company_name,company_post_code,company_id,token=ch_token):
        '''
        Initialise with the access token and the company name 
        '''
        #Initialise the search client
        self.token = token
    
        #Get the company name (from GTR)
        self.name = company_name
        
        #Get the company postcode (from GTR)
        self.post_code = company_post_code
        
        #Get the company ID (from GTR)
        self.id = company_id
    
    @ratelim.patient(550,time_interval=300)
    def query_ch(self):
        '''
        Query CH using the ch wrapper
        '''
        
        #Initialise the search client
        search_client = chwrapper.Search(self.token)
        
        #Search company
        search_results = search_client.search_companies(self.name)
        
        #We either get results of a failure
        
        #If we get a result
        if search_results.status_code == 200:
            self.results = search_results
            self.json = search_results.json()
            self.success = True
        
        else:
            self.results= search_results
            self.json = {'failed_query: {error}'.format(error=search_results.status_code)}
            self.success = False    
    
    def get_best_result(self):
        '''
        Extract results and get postcode matches
        
        '''
        #Only does this if the query was successful
        #This is quite long. Probably shoud create some functions and take out of here
        
        if self.success == True:
            
            #Extract all results
            items = self.json['items']
            
            #If there are any results!
            if len(items)>0:
            
                
                all_results_pc = [[x['address']['postal_code'] 
                                if 'postal_code' in x['address'].keys() else 'no_postal_code',x] for x in items if
                                 x['address']!=None]
                
                #Only returns those results with a match on postcodes
                postcode_matches = [x[1] for x in all_results_pc if x[0]==self.post_code]
                
                #How many matches on postcode do we get?
                self.number_matches = len(postcode_matches)
                
                if len(postcode_matches)>0:
                    #For now, focus on the top result returned by CH <- check_this
                    self.best_match = postcode_matches[0]

                    self.best_ch_number = postcode_matches[0]['company_number']
                    
                else:
                    self.best_match = 'no_matches'
                    self.best_ch_number = 'no_matches'
        
            else:
                self.best_match = 'no_addresses'
                self.best_ch_number = 'no_addresses'
        
        else:
            self.best_match = 'no_data'
            self.best_ch_number = 'no_data'
            
                
    
    def get_ch_data(self,ch,creative_lookup):
        
        '''
        Extract metadata from ch dataset
        
        '''
        
        #If the system hasn't failed, match vs. ch
        
        if self.best_ch_number not in ['no_data','no_matches','no_addresses']:
            
            #Fortunately, the company numbers are strings so we can match without too much hassle
            company_match = ch.loc[ch.CompanyNumber==self.best_ch_number,:]
            self.company_match = company_match
            
            if len(company_match)==0:
                company_match = 'no_ch_metadata'
                self.sic_4 = 'no_ch_metadata'
                self.is_creative = 'no_ch_metadata'
                self.creative_sector = 'no_ch_metadata'
            
            else:
                #self.ch_database_match = company_match
                
                #Extract SIC code
                sic_code = list(company_match['SICCode.SicText_1'])[0]
                
                #Perhaps pre-process this in the ch data above
                
                #There are missing values in the SIC codes
                
                if sic_code != 'None Supplied':
                    
                    #If it is not missing, separate the sector code from the text description
                    sic_number = sic_code.split(" -")[0]
                    
                    #Some of them are 4 digits and some are 5
                    if len(sic_number)<5:
                        self.sic_4 = sic_number
                    else:
                        self.sic_5 = sic_number
                        self.sic_4 = sic_number[:-1]
                        
                    #Now match with CIs:
                    if self.sic_4 in [str(x) for x in ce_lookup.keys()]:
                        self.is_creative = True
                        self.creative_sector = creative_lookup[self.sic_4]
                    else:
                        self.is_creative = False
                        self.creative_sector = 'non_creative'
                
                else:
                    self.sic_4 = 'no_sic'
                    self.is_creative = 'no_sic'
                    self.creative_sector = 'no_sic'
        else:
            self.company_match = self.best_match
            self.sic_4 = self.best_match
            self.is_creative = self.best_match
            self.creative_sector = self.best_match
                    
    def process_all(self):
        '''
        Process all the data with the 3 methods above
        
        '''
        
        self.query_ch()
        self.get_best_result()
        self.get_ch_data()
            

## 1. Load data

In [5]:
#Load Gateway to research projects and organisations
projects = pd.read_csv(ext_data+'/gtr_projects.csv')
orgs = pd.read_csv(ext_data+'/gtr_organisations.csv')

#Addresses
with open(ext_data+'/org_geo.p','rb') as infile:
    org_lat_lng_dict = pickle.load(infile)


In [6]:
#Download CH data dump

ch_dump = "http://download.companieshouse.gov.uk/BasicCompanyDataAsOneFile-2017-07-01.zip"

urllib.request.urlretrieve(ch_dump,ext_data+'/BasicCompanyDataAsOneFile-2017-07-01.zip')

In [314]:
%%!

cd ../data/external/

unzip 'BasicCompanyDataAsOneFile-2017-07-01.zip'


['Archive:  BasicCompanyDataAsOneFile-2017-07-01.zip',
 '  inflating: BasicCompanyDataAsOneFile-2017-07-01.csv  ']

In [6]:
#Load the CSV data

cols_to_keep = ['CompanyName', ' CompanyNumber','RegAddress.PostCode','IncorporationDate',
                'SICCode.SicText_1', 'SICCode.SicText_2', 'SICCode.SicText_3','SICCode.SicText_4',
               'CompanyStatus','CountryOfOrigin']

#Read the data
ch_df = pd.read_csv(ext_data+'/BasicCompanyDataAsOneFile-2017-07-01.csv',usecols=cols_to_keep)
ch_df.columns = [x.strip() for x in ch_df.columns]

In [7]:
#Read DCMS codes
ce_codes = pd.read_csv(ext_data+'/ce_codes.csv')

ce_lookup = {str(x):y for x,y,z in zip(ce_codes.code,ce_codes.label2,ce_codes.type) if z=='SIC'}

## 2. Get Companies House data


**Logic and reasons for missing data**

````````
Extract postcodes and organisation names from the projects df
                            |
                            |-> Some organisations don't have postcodes or are not based in the UK
                            |
    Query CH API with names and deduplicate on postcodses
                            |
                            |-> Some org names don't return any matches
                            |
    Merge matches with CH data dump and extract SIC codes
                            |
                            |-> Some org names aren't in the data dump (they are subsidiaries or dissolved)
                            |   or have not provided SIC codes
                            |
    Tag companies with sector based on SIC code
                            |
                            |
                           Result
                            
```````                                
                                    
                                    

In [8]:

#NB the projects database only includes research grants; the orgs database includes everyone

#We begin with 80K project/org combos, 42K unique project and 6K unique orgs involved in projects
len(projects)
len(set(projects.project_id))
len(set(projects.org_id))

#And 35K organisations 
len(set(orgs.id))

34951

In [37]:
#Work with organisations file

#Step 1.

#Extract postcodes from the org file. There is a field there with a json (text) object containing addresses
#and postcodes (yipee!) for UK companies

#'Is there a single address per company?'
#Yes
print(pd.Series([len(json.loads(x)['address']) for x in orgs['addresses']]).value_counts())


#Now we extract the postcodes. NB in some cases we have no postcodes, in other cases we have no addresses
orgs['postcode'] = [[x['postCode'] if 'postCode' in x.keys() else 'no_postcode' for x in json.loads(add)['address']] for add in 
                  orgs['addresses']]

#Where the list was empty, we return a no_address
orgs['postcode'] = [x[0] if len(x)>0 else 'no_postcode' for x in orgs['postcode']]

#Check some examples where we have no addresses

print(orgs['postcode'].value_counts()[:10])
print(orgs['postcode'].value_counts()[-10:])

no_pc_examples = [json.loads(x)['address'] for x in orgs.loc[orgs['postcode']=='no_postcode','addresses'][:3]]
no_add_examples = orgs.loc[orgs['postcode']=='no_address','name'][:20]

print(no_pc_examples)
print(no_add_examples)

#No postcode examples are outside the UK, no address are a mix of orgs outside the UK and departments and 
#organisations with little data.

#We also have postcodes from outside the UK. 
#Identify organisations outside the UK
orgs['country'] = [[x['country'] if 'country' in x.keys() else 'no_country' for x in json.loads(add)['address']] for add in 
                  orgs['addresses']]
orgs['country'] = [x[0] if len(x)>0 else 'no_country' for x in orgs['country']]

orgs['country'].value_counts().head()

1    24221
0    10730
dtype: int64
no_postcode    14258
CB4 0WS           36
LS2 9DF           31
CB22 3AT          27
N1 7GU            27
TS10 4RF          24
OX11 0QX          23
WA4 4FS           21
EC1V 4PW          21
SA2 8PP           19
Name: postcode, dtype: int64
FK12 5DQ    1
AL2 2RA     1
TQ2 7QL     1
CB4 0QA     1
G15 7SP     1
SE13 7FQ    1
TR1 9GH     1
E14 5LB     1
M33 4DX     1
SN3 6BB     1
Name: postcode, dtype: int64
[[], [], []]
Series([], Name: name, dtype: object)


no_country        26859
United Kingdom     5585
United States       575
Germany             227
France              163
Name: country, dtype: int64

In [41]:
#We have countries with no postcode and postcodes with no country. Weird!
pd.crosstab(orgs['country']!='no_country',orgs['postcode']!='no_postcode')

#It seems that some UK addresses have no country
orgs.loc[orgs.country=='no_country',"addresses"][3]


'{"address": [{"id": "5A58C11D-0031-44CB-9178-65E8C654CABF", "city": "Kidlington", "type": "MAIN_ADDRESS", "line1": "7 Begbroke Science Park\\r\\nSandy Lane\\r\\nYarnton", "county": "Oxfordshire", "region": "South East", "created": 1493118996000, "postCode": "OX5 1PF"}]}'

In [12]:
org_seed_df = orgs.loc[[(x not in ['no_postcode','no_address']) &
                        (y.lower() in ['no_country','united kingdom','uk']) for x,y in 
                        zip(orgs['postcode'],orgs['country'])],['id','name','postcode','country']]

#This is the seed with organisation ids, names and postcodes
org_seed = [[x,y.lower(),z] for x,y,z in zip(org_seed_df['id'],org_seed_df['name'],org_seed_df['postcode'])]

                                     
#~20,000 organisations

19795

In [13]:
with open(int_data+'/gtr_query_results.p','rb') as infile:
    gtr_processing_results = pickle.load(infile)

In [15]:
gtr_update = len(gtr_processing_results)

20693

In [40]:
#Container
#gtr_processing_results = []

#This loops initialises a GTR_org object and processes the data to get company sectors etc. It stores it in
#gtr_processing_results


for num,x in enumerate(org_seed[gtr_update:]):
     
    #Counter
    if num % 100 == 0:
        print('running '+str(num))
    
    successful_run = num
    
    #Initialise
    org = GTR_org(token=ch_token,
                  company_id = x[0],
                  company_name = x[1],
                  company_post_code=x[2])
    
    #Run all the processing
    org.process_all()
    
    if org.success == False:
        print(org.json)
    
    #What do I want to get back? 3 lists.
    #Names, SIC codes, Creative Sectors
    gtr_processing_results.append([org.id,org.name,org.company_match,org.sic_4,org.creative_sector])



In [12]:
#Had to keep updating and re-running the analysis
gtr_update = len(gtr_processing_results)

In [16]:
pd.Series([x[4] for x in gtr_processing_results]).value_counts()

no_matches                                9568
non_creative                              8410
no_ch_metadata                            1102
IT, software and computer services         710
Music, performing and visual arts          215
no_addresses                               174
no_sic                                     102
Design                                      94
Film, TV, video, radio and photography      83
Architecture                                81
Publishing                                  60
Museums, galleries and libraries            53
Advertising and marketing                   41
dtype: int64

In [29]:
with open(int_data+'/gtr_query_results.p','wb') as outfile:
    pickle.dump(gtr_processing_results,outfile)

### Quality assurance and additional data collection

* Identify types of non-matched organisations. Are they universities / charities / government bodies?
* Identify strategies to enrich our dataset
  * Eg look at Charities Commission data?




In [17]:
missing_states = ['no_matches','no_ch_metadata','no_addresses','no_sic']

In [18]:
#What coverage do we get of organisations in the grant data?

matched_ids = [x[0] for x in gtr_processing_results if x[4] not in missing_states]
len(matched_ids)

len(set(projects.org_id_short)- set(matched_ids))

#Not very impressive. Let's check what hasn't been matched

no_match_grants = projects.loc[[x not in set(matched_ids) for x in projects.org_id_short],'name'].value_counts()

#So it is universities
no_match_grants.head()

#Meaning that some universities are in companies house and others aren't.
#This is quite annoying

University College London    3561
University of Cambridge      3072
University of Edinburgh      2578
University of Birmingham     1524
King's College London        1456
Name: name, dtype: int64

In [31]:
orgs_no_match_df =orgs.loc[[x in no_match_grants.index for x in orgs.name],:]

print(orgs_no_match_df['country'].value_counts()[:10])


United Kingdom    3282
no_country        1266
Germany              5
UNITED KINGDOM       3
Canada               3
France               2
Switzerland          2
Australia            2
Belgium              1
Italy                1
Name: country, dtype: int64


In [42]:
#Next - how many are foreign...but provided an address?

#Match with the org df
#orgs_no_match_df = pd.merge(orgs,no_match_orgs,left_on='id',right_on='id')

orgs_no_match_df =orgs.loc[[x in no_match_grants.index for x in orgs.name],:]

#orgs_no_match_df['country'] = [[x['country'].strip().lower() if 'country' in x.keys() else 'no_country' for x in json.loads(add)['address']][0] for add in 
#                  orgs_no_match_df['addresses']]

#Extract country
print(orgs_no_match_df['country'].value_counts()[:10])

#Print no country. 6134 of the no matches have 'no country'... but do they have postcodes?
np.sum([x not in ['no_country','United Kingdom'] for x in orgs_no_match_df['country']])

#This table contains countries and why we couldn't match them. 
status_table = pd.crosstab(orgs_no_match_df['country'],orgs_no_match_df['status'])

#How many are in the 'no_matches' category?
status_table.loc[[x for x in status_table.index if x not in ['no_country','united kingdom']],'no_matches'].sum()

United Kingdom    3282
no_country        1266
Germany              5
UNITED KINGDOM       3
Canada               3
France               2
Switzerland          2
Australia            2
Belgium              1
Italy                1
Name: country, dtype: int64


KeyError: 'status'

876 unmatched countries. Still ~8600 to explain.

In [222]:
#What about the 'no_ch_metadata' (second biggest issue)?

no_ch_data = [x for x in gtr_processing_results if x[4] == 'no_ch_metadata']

#Argh, we didn't save the result from the CH query... or even the companies house name. Fail! <- ADD this for 
#next time. Lesson. Always save the results of an API query somewhere

#Let's run 50 of them and extract their status.
no_ch_seed = [x for x in org_seed if x[0] in [x[0] for x in no_ch_data]]

no_ch_checks = [no_ch_seed[x] for x in np.random.randint(0,len(no_ch_seed),50)]

In [19]:
#Rerun this in the office

no_ch_info = []

#Extract the best match and CH number
for x in no_ch_checks:
        org = GTR_org(token=ch_token,
                  company_id = x[0],
                  company_name = x[1],
                  company_post_code=x[2])
    
        #Run all the processing
        org.process_all()
    
        no_ch_info.append([org.best_ch_number,org.best_match])


In [223]:
no_ch_info_status = pd.Series([x[1]['company_status'] for x in no_ch_info]).value_counts()
print(no_ch_info_status/np.sum(no_ch_info_status))

#Most companies with no CH data appear to be dissolved. What about the exceptions?

open_no_ch_data = [x[1] for x in no_ch_info if x[1]['company_status']!='dissolved']

#All the other no matches are UK establishments of overseas companies.
pd.Series([x['company_type'] for x in open_no_ch_data]).value_counts()

dissolved    1.0
dtype: float64


Series([], dtype: int64)

In [224]:
#What about the 'no results' orgs?
no_addresss_data = [x for x in gtr_processing_results if x[4] == 'no_addresses']
#These ones simply return no CH results
len(no_addresss_data)

174

In [225]:
#What about the no sic orgs?
no_sic_data = [x for x in gtr_processing_results if x[4] == 'no_sic']
#It looks like they never supplied a SIC code

In [226]:
#So it all comes downto the no_matches. Let's check how many of them can we get from the Charities commission
#The significant majority of them appear to be universities/academic institutions and charities. 
#We can go through them in turn.

## 3. Tag universities


**Options**

* Use the grid database

In [256]:
#Load the extract_charity data
#We will use the fuzzywuzzy fuzzy matching package
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


def extract_country_institutions(grid_file,country_list):
    '''
    Extracts contact details for the institutions in a country
    country is a list (we can extract information from more than one country)
    
    
    '''
    
    #List of institutes
    institute_list = grid_file['institutes']
    
    #Some institutes have no address - they are 'redirected'.
    #We don't need their information
    
    institute_w_address = [x for x in grid['institutes'] if 'addresses' in x.keys()]
    
    #Get institutes in country
    institutes_in_country = [x for x in institute_w_address if x['addresses'][0]['country_code'] in country_list]
    
    #Return institutes
    return(institutes_in_country)

def flatten(my_list):
    '''
    Goes through elements in a list and if they are not empty, it extracts the first value. 
    Otherwise it turns values into nans.
    
    '''
    
    flat = [x[0] if len(x)>0 else np.nan for x in my_list]
    return(flat)


In [234]:
with open(ext_data+'/grid.json', 'r') as infile:
    grid = json.load(infile)

In [252]:
#iso code for emirates = AE
uk_institutes = extract_country_institutions(grid,['GB'])

#Get information 
uk_res_df = pd.DataFrame([[x['name'],x['addresses'][0]['city'],
                          x['links'],x['wikipedia_url'],x['types']] for x in uk_institutes],
                       columns=['name','city','url','wikipedia','type'])

#Flatten (ie. extract elements from lists)
uk_res_df['url'] = flatten(uk_res_df['url'])
uk_res_df['type'] = flatten(uk_res_df['type'])

#Here are the UK research institutions
uk_research_institutions = uk_res_df.loc[uk_res_df.type !='Company',:]

#Lower case and stripped name
uk_research_institutions.loc[:,'name'] = [x.lower().strip() for x in uk_research_institutions['name']]
                          

/usr/local/lib/python3.5/site-packages/pandas/core/indexing.py:601: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [278]:
%%time
#We can dedupe on this.

#Let's get matches > 90 for all organisations in the data

#This is the seed list of organisations we want to run vs Grid and Charity Commission data
orgs_no_match_seed = [[a,b,c] for a,b,c,d,e in zip(orgs_no_match_df['id'],
                                              orgs_no_match_df['name'],
                                              orgs_no_match_df['postcode'],
                                                  orgs_no_match_df['status'],
                                                  orgs_no_match_df['country']) if
                      d == 'no_matches' and e in ['no_country','united kingdom']]

#These are the candidate organisations from Grid
candidates = list(uk_research_institutions['name'])

#These are the matches (if they happen!)
grid_matches = []

#For each name
for num,x in enumerate(orgs_no_match_seed):
    
    if num % 500==0:
        print(num)
    
    #Extract matches
    matches = process.extractBests(x[1],candidates)
    
    #Get valid matches
    valid_match = matches[0] if matches[0][1]>90 else 'no_good_match'
    
    grid_matches.append([x[0],x[1],valid_match])
    

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
CPU times: user 6h 56min 33s, sys: 2min 15s, total: 6h 58min 48s
Wall time: 17h 35s


In [281]:
with open(int_data+'/grid_matches.p','wb') as outfile:
    pickle.dump(grid_matches,outfile)

In [290]:
np.sum([x[2]!='no_good_match' for x in grid_matches])
#We got 639 matches here. Let's see what they are.

grid_matches_df = pd.DataFrame([[x[0],x[1],x[2][0],x[2][1]] for x in grid_matches if x[2]!='no_good_match'],
                              columns=['org_id','gtr_name','grid_name','score'])


grid_matches_df.iloc[:,:50]
#Assumption - these ones will have a massive presence in GTR. Let's see.

,org_id,gtr_name,grid_name,score
0,AEBFFA9B-9B1E-4235-96E2-ACB761119425,South West College Omagh,south west college,95
1,9D2232B9-DC18-4507-96F6-63DAE75B0284,Royal Pharmaceutical Society of Great Br,royal pharmaceutical society,95
2,C9E3BB74-B3E7-4454-9C35-8EB1373856B7,Association for the Conservation of Energy,association for the conservation of energy,100
3,58451366-6877-4B7D-B2D4-8D0416BA6DFD,The Royal Institute of Navigation,royal institute of navigation,95
4,5BB4F8BF-B4E0-4EAF-9AF5-885E19D64850,University of Strathclyde,university of strathclyde,100
5,289B3363-B06A-4E83-9F57-80E81DCB0DA2,University Hospital Southampton NHS Foundation...,university hospital southampton nhs foundation...,100
6,9152582B-272B-4BE3-8223-A92E94DF0D75,Department of Agriculture & Rural Development ...,department of agriculture and rural development,91
7,440BEB68-030A-41A9-924E-02DCF0DD71D4,The Ideas Foundation,ideas foundation,95
8,09B4D85F-B21E-4FB5-A591-07DCB5E231B0,Scottish Universities Physics Alliance,scottish universities physics alliance,100
9,7C771F1D-4232-4D26-BB06-09A91AE633E5,The Arts Catalyst,arts catalyst,95


In [291]:
#35% of all projects
np.sum([x in set(grid_matches_df['org_id']) for x in projects['org_id_short']])/len(projects)


0.34882462498741568

In [298]:
#We can combine this with the grid_data to determine the type of organisation
#Later, we can remove these from the GTR data.
grid_matches_meta_df = pd.merge(grid_matches_df,uk_research_institutions,left_on='grid_name',right_on='name')

#These are the types of organisations in the data
grid_matches_meta_df['type'].value_counts()

Healthcare    157
Government    154
Nonprofit     106
Education      85
Other          64
Facility       52
Archive        26
Name: type, dtype: int64

In [300]:
#We will continue extracting information about these
orgs_no_match_seed_2 = [x for x in orgs_no_match_seed if x[0] not in set(grid_matches_meta_df['org_id'])]

len(orgs_no_match_seed_2)

8053

## 4. Get Charities commission data

Alas, the API seems to be down. I am using the data extract downloaded from here:

http://data.charitycommission.gov.uk/default.aspx

And extracted into CSVs from here: https://github.com/ncvo/charity-commission-extract

And following these instructions.

https://github.com/ncvo/charity-commission-extract

**Steps to follow**

* Load extract_charity_data (contains name and address). Do fuzzy matching on that via name/postcode.
* Use the outputs to query  extract_class. This also requires using extract_class_ref to analyse the codes



In [301]:

#Load. NB I had to fix the headers; there was an issue with the formating that was creating some errors.
charities = pd.read_csv(ext_data+'/charity_commission_extract/extract_charity_changed.csv',encoding='latin-1')
charities = charities.loc[:,[x for x in charities.columns if 'Unnamed' not in x]]

#Now we want to deduplicate the no org matches vs this list.

activities = pd.read_csv(ext_data+'/charity_commission_extract/extract_class.csv')
activities_codes = pd.read_csv(ext_data+'/charity_commission_extract/extract_class_ref.csv')
#objects = pd.read_csv(ext_data+'/charity_commission_extract/extract_objects.csv',error_bad_lines=False)


/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,6,8,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [302]:
#Lowercase clean names
charities['name'] = [str(x).lower().strip() for x in charities['name']]
charities['postcode'] = [str(x).strip() for x in charities['postcode']]

#Turn activities df into a lookup
activities_codes.dropna(axis=0,inplace=True)


activities_lookup = {x:y.lower() for x,y in zip(activities_codes['classno'],activities_codes['classtext'])}

In [303]:
#The strategy we will follow is to match on postcodes and then fuzzy match.
#How many postcodes do we have?
print(np.sum(charities.postcode.isnull()==False))

#What will be the strategy?

#For each variable in the GTR data, subset the charities data by postcode and then select best matches on name.


class GTR_org_charity():
    '''
    We use this class to fuzzy match organisation names in the GTR unmatched orgs list
    and organisations in the charities commission data extract.
    #Steps:
    1. subset charities df by postcode.
    2. get best matches by name (with fw)
    3. Extract their sectors.
    
    '''

    def __init__(self, char_df,char_activities,activities_lookup):
        '''
        Initialise
        
        '''
        
        #Initialise
        self.char_data = char_df
        self.activities = char_activities
        self.class_lookup = activities_lookup
        
    
        
    def fuzzy_match(self,org_name,org_postcode,thres=90):
        '''
        Steps as above
        
        '''
        
        #Load variables
        char_data = self.char_data
        self.name = org_name.lower()
        self.postcode = org_postcode

        
        #We use the halt variable to return no matches etc.
        halt = None
        
        #Subset by postcode match
        charities_in_postcode = char_data.loc[char_data['postcode']==self.postcode,:]
        
        #Extract names
        self.potential_matches = list(charities_in_postcode['name'])
        
        #Store the number of potential matches
        self.potential_matches_n = len(self.potential_matches)
        
        
        if self.potential_matches_n==0:
            
            #Halt 
            halt = 'no_orgs_in_pc'
            self.strong_matches = halt
            self.top_match = halt
            self.top_match_data = halt
            self.top_match_score = halt
        
        else:
            
            #Give me the best matches based in names
            top_matches = process.extractBests(self.name,self.potential_matches,limit=3)

            self.strong_matches = top_matches

            #Select the top match (if there is one with a score above the threshold)
            self.top_match = top_matches[0][0] if top_matches[0][1]>thres else 'no_good_matches'
            self.top_match_score = top_matches[0][1] if top_matches[0][1]>thres else 'no_good_matches'
            
            
            #Halt if no good matches
            
            if self.top_match == 'no_good_matches':
                halt = 'no_good_matches'
                self.top_match = halt
                self.top_match_data = halt
                self.top_match_score = halt
                
                
            
            else:
                self.top_match_data = charities_in_postcode.loc[charities_in_postcode['name']==self.top_match,:]
        
        self.halt = halt
                
                
    def label_class(self):
        '''
        Labels a charity with the classes where it works
        
        '''
        
        #We will only extract the classes if we obtained a valid result
        if self.halt not in ['no_good_matches','no_orgs_in_pc']:
            
            #Match top match with charity activities
            merged = pd.merge(self.top_match_data,self.activities,left_on='regno',right_on='regno')
            
            #This adds a column with activity names
            merged['activity_name'] = [self.class_lookup[x] for x in merged['class']]
            
            #Group by charity and create a list of activity names
            grouped_activities = merged.groupby('regno')['activity_name'].apply(lambda x: list(x))
            
            #Out
            self.activities = list(grouped_activities)
        else:
            self.activities = self.halt
                                       

317583


In [305]:
%%time

#Do the fuzzy matching on the second org seed
#These are the matches
char_out = []

for x in orgs_no_match_seed_2:
    
    #Initialise and run
    fuzz = GTR_org_charity(charities,activities,activities_lookup)               
    fuzz.fuzzy_match(x[1],x[2])
    fuzz.label_class()
    
    
    
    #Output results
    results = [x[0],x[1],fuzz.top_match,fuzz.top_match_data,fuzz.top_match_score,fuzz.activities]
    char_out.append(results)

CPU times: user 3min 38s, sys: 3.9 s, total: 3min 42s
Wall time: 3min 46s


In [27]:
#Good charity matches
char_matched = pd.DataFrame([[x[0],x[1],x[2],x[4]] for x in char_out if x[1] not in ['no_orgs_in_pc',
                                                                         'no_good_matches']],
                           columns=['gtr_id','gtr_name','cc_name','score'])

#Sadly, only around 100. The quality of matches degrades a lot if we go below 90 in the score. A brute 
#force approach would take too much time.

char_matched.head()
char_matched.tail()

#Extract matched ids
char_matched_ids = list(char_matched['gtr_id'])


## 4. Manual matching

Finally, we run some manual checks using keywords. This is a somewhat crude approach we will need to refine further down the line


In [28]:

orgs_no_match_char = [x for x in orgs_no_match_seed if x not in char_matched_ids]

#Check how many of these have university / college / library / council / bbc


keywords = ['university','college','library','museum','council','bbc','arts council','design council',
           'department','agancy','hospital','nhs']

manual_class = [[x[0],x[1],[z for z in keywords if z in x[1].lower()]] for x in orgs_no_match_char]

manual_list = [x for x in manual_class if len(x[2])>0]

print(len(manual_list))

pd.Series([x[2][0] for x in manual_list]).value_counts()

928


university    318
council       232
nhs           118
hospital       96
museum         67
college        65
department     21
library        10
bbc             1
dtype: int64

In [29]:
#So at the end of this process we have ~ 7764 organisations with no matching.

un_matched = [x for x in manual_class if len(x[2])==0]

len(un_matched)


un_matched[:20]

#Still seems to contain a significant number of charities and non-UK organisations. We are potentially also
#missing out on subsidiaries of organisations with a different address. There are all sorts of things we could
#do to address this but now it's not the time.

[['C3962A50-B80D-4615-A2E1-777CDD9A894B', 'IAgrE', []],
 ['E9205A67-5D1C-41EC-9E91-609012855731', 'Lancashire One Limited', []],
 ['FB1C557F-EC19-4AB2-8F6F-617442B980B4', 'RJC Dance', []],
 ['FBF5DD26-8F4F-419D-A165-78F8ECB28FE9', 'Prima Bakeries Ltd', []],
 ['9D2232B9-DC18-4507-96F6-63DAE75B0284',
  'Royal Pharmaceutical Society of Great Br',
  []],
 ['A15F46CD-89B2-46F4-8048-86F97E998DC5',
  'The Ambassador Theatre Group Limited',
  []],
 ['A285F063-8D46-4333-AEA8-66C566540929', 'eCommera', []],
 ['9E60AE6A-7453-4AA6-A466-A8A764B0D793', 'Cherry Valley Farms Ltd', []],
 ['9F758012-007F-4A1C-B4F2-AD1A1E58F153', 'Aeguana Ltd', []],
 ['9FA8B077-3158-4314-936E-6E90F57FC91A', 'Traceall Global Limited', []],
 ['A00D6D17-9648-4CB0-8729-6D224CD01652', 'Sustanable Technology Ltd', []],
 ['A7B28613-75F6-49B3-83AD-65E962277B2F',
  'Computerised Information Technology Limited',
  []],
 ['C7FFCC23-E2D4-4940-811E-8D658D6A6C50', 'Western Infirmary, Glasgow', []],
 ['C9E3BB74-B3E7-4454-9C35-8EB137385

## 4. Analysis

**Questions**


* What are the top research disciplines for the creative industries?

* What are the top sectors active in those disciplines

* What is the geography of collaboration between universities and organisations in other disciplines

**Actions**

* Load the classes I used to label projects with disciplines
* Crosstab project disciplines vs creative status
* Identify 'creative disciplines'
* 




In [80]:
#Classes

#Functions used to get topics below
def get_max_topics(df):
    
    #This extracts one topic for each project and turns it into a df
    topic_top = pd.DataFrame([{x:1} for x in df.apply(lambda x: x.argmax(),axis=1)])
            
    #Fill the nas with zeroes
    topic_top.fillna(value=0,inplace=True)
    
    return(topic_top)

def get_thres_topics(df,value):
    #This creates a binary matrix where topics with prob>value = 1, else zero.
    topic_bin = df.applymap(lambda x: 1 if x>value else 0)
    
    return(topic_bin)
    
def get_top_topics(df,value):
    '''
    This creates a binary matrix where each project has a 1 in a topic if it scores above a certain threshold
    This also takes the keyword argument value
    Will this take ages?
    '''
    lead_topics = [df.iloc[row,:].sort_values(
        ascending=False).index[:value] for row in np.arange(0,len(df))]
            
    #Create a df using similar syntax as before
    topic_lead = pd.DataFrame([{x:1 for x in el} for el in lead_topics])
    topic_lead.fillna(value=0,inplace=True)
    return(topic_lead)        

def get_lq(dist_df):
    """
    Function to produce location quotients based on a contingency table (e.g. geographical distribution of activity)
    Assumes that rows = observations (e.g. places) and columns = variables (e.g. industry, sector of activity)
    
    #Input:
    #A contingency table (output of pd.crosstab(unit,variable))
    #Returns a table with location quotients (and one variable with labels for plotting)
    """
    column_counts = dist_df.sum(axis=1).reset_index()
    
    props =  dist_df.apply(lambda x: x/dist_df.sum(axis=1))
    lqs = props.apply(lambda x: x/(dist_df.sum(axis=0)/dist_df.sum().sum()),axis=1).reset_index()
    
    lqs_w_counts = pd.merge(lqs,column_counts,on=lqs.columns[0])
    lqs_w_counts.rename(columns={0:"total_projects"},inplace=True)
    
    return(lqs_w_counts)

class GtR_location_analysis():
    '''
    This class analyses the levels of activity in a topic community in an area.
    To do this, it allocates projects to topic communities based on their weights
    It then groups those by area and generates stats
    We need to decide whether the focus is on:
        #projects led by organisations in the area
        #projects involving organisations in the area
        #organisations in the area 
    
    '''
    
    def __init__(self,projects_df):
        '''
        Initialise the class with a project_df
        
        '''
        #Load the projects
        self.projects_df =  projects_df
        
        #Create a topic communities attribute with their labels (we can use it for subsetting)
        self.topic_comms = [x for x in self.projects_df.columns if x.split("_")[0]=='tc']
        
        #Create a topics_df attribute
        #self.topics_df = projects_df[self.topic_comms]
        
        #Create a metadata attribute (with relevant info for each org-project pair)
        self.meta_data_df = self.projects_df[list(set(self.projects_df.columns)-set(self.topic_comms))]
        
        #Create an org_geo coded df
        #self.org_geo_df = projects_df.drop_duplicates('org_id')[['org_id','nation','LAD13NM']].set_index('org_id')
        
        
        
    def classify_in_topics(self,how='max',**kwargs):
        '''
        Here, we convert the probabilities in the topic df into 1/0s for counting later. 
        We will have the following modalities:
            -max: Project is allocated to the topic where it has its maximum probability
            -thres: Any project with prob >thres (which could be zero) =1
            -top_x: Any project with topic in its top x has a 1 in that topic.
        
        '''
        
        #Create a df with topics to process
        topics = self.projects_df[self.topic_comms].copy()
        
        #We will input a kwargs dict with keys for the methods. We will need to tell the other methods
        #which ones to generate.
        #We put the outputs a an allocated_projects attribute which also includes the metadata.
        
        if how=='max':
            self.allocated_projects= pd.concat([self.meta_data_df,get_max_topics(topics)],axis=1)
            
        elif how=='thres':
            #This takes the keyword argument value
            
            self.allocated_projects = pd.concat([self.meta_data_df,get_thres_topics(topics,kwargs['param'])],axis=1)

            
        elif how=='top':
            self.allocated_projects = pd.concat([self.meta_data_df,get_top_topics(topics,kwargs['param'])],axis=1)
             
    
    def generate_area_stats(self,area='LAD13NM',unit='org',role='LEAD_ORG',stat='count',subset_years=None):
        '''
        This function generates project location stats. 
        If we give it the org unit parameter, it returns the stat of organisations working in an area.
        Otherwise, it returns the stat projects in the area. 
        The stat can be number of projects or total funding
        
        '''
        
        #Extract the allocated topics and metadata
        
        allocated_projects = self.allocated_projects.copy()
        topic_comms = self.topic_comms
        
        #metadata = self.meta_data_df.copy() 
        #geo_org = self.org_geo_df.copy()
        
        if subset_years != None:
            #Subset both dataframes by the start years (a list provided as input)
            #metadata = metadata.ix[[x in subset_years for x in metadata.start_year],:]
            #allocated_topics = allocated_topics.ix[metadata.index,:]
            allocated_projects = allocated_projects[[x in subset_years for x in allocated_projects.start_year]]
            
        #If we are looking at orgs, it's relatively simple:
        if unit  == 'org':
            
            #(i.e. is the organisation active in a topic cluster or not?)
            org_activity = allocated_projects.groupby(['org_id',area])[topic_comms].max().reset_index(drop=False)
            org_activity.drop('org_id',axis=1,inplace=True)
            
            #Group over the area and extract
            output = org_activity.groupby(area).sum()
                                
        if unit == 'proj':
            #Do we want to focus on projects led by organisations in the area, or projects involving organisations
            #in the area?
            
            if role=='LEAD_ORG':
                #Drop all non lead orgs
                allocated_projects = allocated_projects.ix[allocated_projects.role=='LEAD_ORG',:]
            else:
                #Drop all project duplicates in each LAD
                allocated_projects = allocated_projects.groupby(
                    area).apply(lambda x: x.drop_duplicates('project_id')).reset_index(drop=True)
        
            if stat == 'count':
                #Count number of projects with some activity in the area
                area_counts = allocated_projects.groupby(area)[topic_comms].sum()
                output = area_counts
                
            if stat == 'funding':
                #Extract budgets for projects in different topic clusters (NB if we have more than one
                    #label for cluster, there will be double counting.)
                funds = allocated_projects[topic_comms].apply(lambda x: x*allocated_projects['value_pounds'])
                
                funds[area] = allocated_projects[area]
                
                #Sum funds over locations
                area_funds = funds.groupby(area)[topic_comms].sum()
                
                output = area_funds
        
        #Relabel columns (remove tc_)
        output.columns = [x[3:] for x in output.columns]
        self.area_levels = output
        
        #Generate other stats
        #Shares of sector in area
        self.area_topic_shares = output.apply(lambda x: x/output.sum(axis=1),axis=0)
    
        #Extract LQs
        lq_df = get_lq(output).set_index(output.index.name)
        lq_df.fillna(value=0,inplace=True)
        self.area_topic_lq = lq_df.ix[:,:-1]
        
        #Share of area in sector in the UK
        self.area_share_in_topic =  output/output.sum()

In [30]:
#Load the project

#Here they are.
projects_comms = pd.read_csv(ext_data+'/3_08_2017_projects_topic_communities.csv')

In [121]:
#Now we want to take these projects and determine the creative specialisations.

#Crude strategy

org_sector = pd.DataFrame([[x[0],x[1],x[3],x[4]] for x in gtr_processing_results if x[4].split("_")[0]!='no'],
                          columns=['id','name','sic','sector'])
org_sector['is_creative'] = ['creative' if x !='non_creative' else 'non_creative' for x in org_sector['sector']]

#Remove University of Portsmouth investments, which appears as an 'advertising' organisation
org_sector_clean = org_sector.loc[org_sector.id!="955C55E8-783E-4842-BB2C-2D275A3CAF82",:]


#Match on org_id (nb we use the one without the GTR Url)

projects_comms_sector = pd.merge(projects_comms,org_sector_clean,left_on='org_id_short',right_on='id')


pd.crosstab(projects_comms_sector['sector'],projects_comms_sector['discipline_allocation'])

discipline_allocation,Arts and Humanities,Engineering and Technology,Environmental Sciences,Life Sciences,Mathematics and Computing,Medical Science,Physics,Social Sciences
sector,,,,,,,,
Advertising and marketing,1,2,0,0,0,0,0,3
Architecture,7,12,2,0,1,0,1,2
Design,7,4,1,0,4,0,0,0
"Film, TV, video, radio and photography",13,0,0,0,4,0,1,2
"IT, software and computer services",2,38,3,0,63,0,0,3
"Museums, galleries and libraries",97,7,64,9,13,0,23,14
"Music, performing and visual arts",146,2,3,0,12,0,1,27
Publishing,19,1,3,2,6,0,0,11
non_creative,2056,7361,2645,3490,3456,3013,1317,2580


In [122]:
creative_ct = pd.crosstab(projects_comms_sector['is_creative'],projects_comms_sector['discipline_allocation'])

creative_ct/creative_ct.sum()


discipline_allocation,Arts and Humanities,Engineering and Technology,Environmental Sciences,Life Sciences,Mathematics and Computing,Medical Science,Physics,Social Sciences
is_creative,,,,,,,,
creative,0.124361,0.008886,0.027931,0.003142,0.028941,0.0,0.01936,0.023467
non_creative,0.875639,0.991114,0.972069,0.996858,0.971059,1.0,0.98064,0.976533


In [123]:
#What are the creative topics?

creative_analysis = GtR_location_analysis(projects_comms_sector)

creative_analysis.classify_in_topics(how='top',**{'param':1})


In [138]:
#Check top topics by sector
classified_projects = creative_analysis.allocated_projects

sector_discipline_analysis = pd.melt(classified_projects.loc[:,['name_x','sector','is_creative','sic']+
                                                             [x for x in classified_projects.columns if
                                                             'tc_' in x]],
                                    id_vars=['name_x','sector','is_creative','sic'])

sector_discipline_analysis_2 = sector_discipline_analysis.loc[sector_discipline_analysis.value>0,:]

#Table with results
table = pd.crosstab(sector_discipline_analysis_2.is_creative,sector_discipline_analysis_2.variable).T.sort_values('creative',
                                                                                                         ascending=False)

#Table normalised by totals in the row
table.apply(lambda x: x/x.sum(),axis=1).sort_values('creative',ascending=False)


table

is_creative,creative,non_creative
variable,,
tc_media_design_creative,196,934
tc_systems_devices,53,810
tc_systems_infrastructure,49,2077
tc_ecosystems,43,799
tc_literature,29,415
tc_museums_exhibitions,24,52
tc_sociology,18,448
tc_history,17,165
tc_instrumentation_probes_measurement,13,81


In [147]:
#It's quite surprising how many non-creative organisations are e.g. participating in media and cultural projects.
#Let's check who they are

sector_discipline_org_analysis = pd.melt(classified_projects.loc[:,['name_x','sector','sic','is_creative']+
                                                             [x for x in classified_projects.columns if
                                                             'tc_' in x]],
                                    id_vars=['name_x','sector','sic','is_creative'])

sector_discipline_org_analysis.loc[(sector_discipline_org_analysis.sector=='non_creative') &
                                  (sector_discipline_org_analysis.variable=='tc_sociology') &
                                   ([all(z not in x.lower() for z in ['university','museum','college'])
                                     for x in sector_discipline_org_analysis.name_x]) & 
                                   ([x[:2]!='85' for x in sector_discipline_analysis.sic]) &
                                   (sector_discipline_org_analysis.value>0),:]

#Lots of academic institutions! We need to take them out.

,name_x,sector,sic,is_creative,variable,value
1849755,Institute of Development Studies,non_creative,7490,non_creative,tc_sociology,1.0
1851686,Marches Energy Agency,non_creative,6399,non_creative,tc_sociology,1.0
1852432,Liverpool School of Tropical Medicine,non_creative,7220,non_creative,tc_sociology,1.0
1854405,National Centre for Social Research,non_creative,7220,non_creative,tc_sociology,1.0
1854408,National Centre for Social Research,non_creative,7220,non_creative,tc_sociology,1.0
1854462,Institute of Food Research,non_creative,7219,non_creative,tc_sociology,1.0
1855939,Barrow Cadbury Trust,non_creative,8899,non_creative,tc_sociology,1.0
1856255,Genesis Leeds,non_creative,8690,non_creative,tc_sociology,1.0
1856256,UK Network of Sex Work Projects,non_creative,9499,non_creative,tc_sociology,1.0
1856896,Single Parent Action Network,non_creative,9499,non_creative,tc_sociology,1.0


In [145]:
sector_discipline_analysis_no_ed = sector_discipline_analysis.loc[
    (sector_discipline_analysis.value>0) & 
    ([all(z not in x.lower() for z in ['university','museum','college'])
                                     for x in sector_discipline_org_analysis.name_x]) 
    & [x[:2]!='85' for x in sector_discipline_analysis.sic],:]

#Table with results
table_no_ed = pd.crosstab(sector_discipline_analysis_no_ed.is_creative,sector_discipline_analysis_no_ed.variable).T.sort_values('creative',
                                                                                                         ascending=False)

#Table normalised by totals in the row
table_no_ed.apply(lambda x: x/x.sum(),axis=1).sort_values('creative',ascending=False)



is_creative,creative,non_creative
variable,,
tc_media_studies,1.000000,0.000000
tc_literature,0.814815,0.185185
tc_philosophy,0.800000,0.200000
tc_museums_exhibitions,0.800000,0.200000
tc_performance,0.647059,0.352941
tc_history,0.608696,0.391304
tc_education,0.500000,0.500000
tc_public_engagement,0.500000,0.500000
tc_media_design_creative,0.466667,0.533333


In [148]:
#Ok - so we need to remove / label universities and educational institutions with a significant presence here.

In [131]:
#This process will not be trivial. We will have to:


#1. Query each organisation name vs companies house,
#2. Deduplicate on postcodes and extract CH number
#3. Get the SIC code from the CH data dump

#Step 1.

#Extract postcodes from the org file. There is a field there with a json (text) object containing addresses
#and postcodes (yipee!) for UK companies

#'Is there a single address per company?'
#Yes
print(pd.Series([len(json.loads(x)['address']) for x in orgs['addresses']]).value_counts())

#Now we extract the postcodes. NB in some cases we have no postcodes, in other cases we have no addresses
orgs['postcode'] = [[x['postCode'] if 'postCode' in x.keys() else 'no_postcode' for x in json.loads(add)['address']] for add in 
                  orgs['addresses']]

#Where the list was empty, we return a non-UK
orgs['postcode'] = [x[0] if len(x)>0 else 'no_address' for x in orgs['postcode']]

#Check some examples where we have no addresses

print(orgs['postcode'].value_counts()[:10])

no_pc_examples = [json.loads(x)['address'] for x in orgs.loc[orgs['postcode']=='no_postcode','addresses'][:3]]
no_add_examples = orgs.loc[orgs['postcode']=='no_address','name'][:20]

#print(no_pc_examples)
#print(no_add_examples)

#No postcode examples are outside the UK, no address are a mix of orgs outside the UK and organisations 

1    24221
0    10730
dtype: int64
no_address     10730
no_postcode     3528
CB4 0WS           36
LS2 9DF           31
CB22 3AT          27
N1 7GU            27
TS10 4RF          24
OX11 0QX          23
EC1V 4PW          21
WA4 4FS           21
Name: postcode, dtype: int64


In [147]:
#Now we want to create a df with organisation names and postcodes to be run vs CH

#org_seed_df = orgs.loc[[x for x in orgs['id'] if x in set(projects['id'])] 
#                        and [x not in ['no_postcode','no_address'] for x in orgs['postcode']],:]


org_seed_df = orgs.loc[[x not in ['no_postcode','no_address'] for x in orgs['postcode']],['id','name','postcode']]

#This is the seed with organisation ids, names and postcodes
org_seed = [[x,y.lower(),z] for x,y,z in zip(org_seed_df['id'],org_seed_df['name'],org_seed_df['postcode'])]

#20,000 organisations

20693

In [303]:
#Rate limitation based on CH T&C
@ratelim.patient(300,time_interval=300)
def extract_org_data(comp_name):
    
    '''
    This function takes a company name and queries it against CH using the chwrapper search client.
    It returns 
    '''
    
    #Output
    
    out= search_client.search_companies(comp_name)
    try:
        #Json output
        returned = out.json()

        #Extract the postcodes for all results to simplify processing later

        #Items in the returned call
        items = returned['items']

        #This is a list where the first element is the postcode and the second element the rest of the results
        items_postcodes = [[x['address']['postal_code'] 
                            if 'postal_code' in x['address'].keys() else 'no_postal_code',x] for x in items]

        return([out,returned,items_postcodes])
    except:
        return([out,'failed'])
        


In [304]:
#Now: we query
#What strategy do we use?

#Initialise the search client
search_client = chwrapper.Search(access_token=ch_token)


#Loop to extract results
#Some observations and gotchas:
    #We obtain 20 results per query. In some cases, the right match might be lower in the list.
    #Let's extract the top 20 for 200 companies and see how matches we have

ch_gtr_results = []

for index in np.arange(len(org_seed)):
    
    #Counter
    if index %500 == 0:
        print(index)
    
    #Extract company name
    comp_name = org_seed[index][1]
    
    #Run query
    query_results = extract_org_data(comp_name)
    
    ch_gtr_results.append(query_results)
    

0


KeyboardInterrupt: 

In [275]:
match = [[result for result in res if result[0]==pc] for res,pc in zip([x[2] for x in ch_gtr_results],[x[2] for x in org_seed[:10]])]

In [287]:
ch_gtr_results[len(ch_gtr_results)-1][0].headers

{'Content-Length': '12075', 'Content-Type': 'application/json', 'Pragma': 'no-cache', 'X-Ratelimit-Reset': '1501233980', 'X-Ratelimit-Limit': '600', 'X-Ratelimit-Remain': '476', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Access-Control-Expose-Headers': 'Location,www-authenticate,cache-control,pragma,content-type,expires,last-modified', 'Date': 'Fri, 28 Jul 2017 09:25:47 GMT'}

In [293]:
ch_gtr_results[123][0].headers

{'Content-Length': '12075', 'Content-Type': 'application/json', 'Pragma': 'no-cache', 'X-Ratelimit-Reset': '1501233980', 'X-Ratelimit-Limit': '600', 'X-Ratelimit-Remain': '476', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Access-Control-Expose-Headers': 'Location,www-authenticate,cache-control,pragma,content-type,expires,last-modified', 'Date': 'Fri, 28 Jul 2017 09:25:47 GMT'}

In [149]:
len(ch_gtr_results)

NameError: name 'ch_gtr_results' is not defined

0

50.0

1200